复制list文件到指定dir

In [ ]:
import shutil
import os

# 设定源目录和目标目录
source_dir = '/Users/naoj306/Desktop/find/slice_new_wcs/'
target_dir = '/Users/naoj306/Desktop/find/slice_new_wcs_list_turn/'

# 确保目标目录存在
os.makedirs(target_dir, exist_ok=True)

# 读取文件编号列表
with open('list_turn.txt', 'r') as file:
    file_numbers = file.read().split()

# 遍历文件编号，复制文件
for number in file_numbers:
    filename = f'12CO_x{number}_updated.fits'
    source_path = os.path.join(source_dir, filename)
    target_path = os.path.join(target_dir, filename)

    # 检查文件是否存在，然后复制
    if os.path.exists(source_path):
        shutil.copy(source_path, target_path)
        print(f'Copied {filename}')
    else:
        print(f'File not found: {filename}')


转移list文件去指定dir

In [ ]:
import shutil
import os

# 定义源目录和目标目录
source_dir = '/Users/naoj306/Desktop/find/slice_new_wcs_diff_1-99_fil_3-3'  # 替换为你的源目录路径
target_dir = '/Users/naoj306/Desktop/find/slice_new_wcs_diff_1-99_fil_3-3/error'  # 替换为你的目标目录路径

# 确保目标目录存在
os.makedirs(target_dir, exist_ok=True)

# 读取文件编号列表
with open('list_error.txt', 'r') as file:
    file_numbers = file.read().split()

# 遍历文件编号，移动文件
for number in file_numbers:
    filename = f'12CO_x{number}_slice_diff_updated.fits'
    source_path = os.path.join(source_dir, filename)
    target_path = os.path.join(target_dir, filename)

    # 检查源文件是否存在
    if os.path.exists(source_path):
        shutil.move(source_path, target_path)
        print(f'Moved {filename} to {target_dir}')
    else:
        print(f'File not found: {filename}')


plot sch=360 1*4 vy slice: origin-median-diff-diffmedian

In [2]:
from astropy.io import fits
import numpy as np
from scipy.ndimage import generic_filter
import os
from astropy.wcs import WCS
import matplotlib.pyplot as plt

def nanmedian_filter(data):
    return np.nanmedian(data)

def generate_and_save_yv_slices_for_specific_x(fits_name, output_path, x_index):
    with fits.open(fits_name) as hdul:
        data = hdul[0].data
        header = hdul[0].header
    wcs = WCS(header)

    # 转换速度单位到km/s
    v_size = header['NAXIS3']
    v_crpix = header['CRPIX3']
    v_crval = header['CRVAL3'] / 1000.0  # m/s to km/s
    v_cdelt = header['CDELT3'] / 1000.0  # m/s to km/s
    velocities = (np.arange(v_size) - (v_crpix - 1)) * v_cdelt + v_crval

    # 获取分子线信息
    molecule = header.get('LINE', 'Unknown Molecule')

    ra_deg = wcs.wcs_pix2world([[x_index, 0, 0]], 0)[0][0]

    dec_pixels = np.arange(data.shape[1])
    ra_pixels = np.full_like(dec_pixels, x_index)
    world_coords = wcs.all_pix2world(np.vstack([ra_pixels, dec_pixels, np.zeros_like(dec_pixels)]).T, 0)
    dec_deg = world_coords[:, 1]  # 获取赤纬度数

    # 获取原始数据切片
    data_slice = data[:, :, x_index].T  # 转置以匹配绘图的轴向

    # 应用1x50 median filter，并自动忽略NaN
    filtered_data = generic_filter(data_slice, nanmedian_filter, size=(50, 1))
    diff_data = data_slice - filtered_data

    # 应用3x3 median filter到diff数据
    filtered_diff_data = generic_filter(diff_data, nanmedian_filter, size=(3, 3))

    fig, axs = plt.subplots(1, 4, figsize=(28, 6), sharex=True)
    fig.subplots_adjust(wspace=0.1, left=0.03, right=0.99, top=0.9, bottom=0.1)

    # 原始数据图像
    im1 = axs[0].imshow(data_slice, origin='lower', aspect='auto',
                        extent=[velocities[0], velocities[-1], dec_deg.min(), dec_deg.max()],
                        cmap='nipy_spectral', vmin=-1, vmax=9)
    axs[0].set_title(f"Original Data at RA {ra_deg:.4f} degree")
    axs[0].set_xlabel("Velocity (km/s)")
    axs[0].set_ylabel("Dec (degrees)")
    cbar1 = fig.colorbar(im1, ax=axs[0], orientation='vertical', label='Temperature (K)')

    # 滤波后的数据图像
    im2 = axs[1].imshow(filtered_data, origin='lower', aspect='auto',
                        extent=[velocities[0], velocities[-1], dec_deg.min(), dec_deg.max()],
                        cmap='nipy_spectral', vmin=-1, vmax=9)
    axs[1].set_title(f"Filtered Data at RA {ra_deg:.4f} degree")
    axs[1].set_xlabel("Velocity (km/s)")
    axs[1].set_ylabel("Dec (degrees)")
    cbar2 = fig.colorbar(im2, ax=axs[1], orientation='vertical', label='Temperature (K)')

    # 差异图像
    im3 = axs[2].imshow(diff_data, origin='lower', aspect='auto',
                        extent=[velocities[0], velocities[-1], dec_deg.min(), dec_deg.max()],
                        cmap='seismic', vmin=-6, vmax=6)
    axs[2].set_title(f"Difference Data at RA {ra_deg:.4f} degree")
    axs[2].set_xlabel("Velocity (km/s)")
    axs[2].set_ylabel("Dec (degrees)")
    cbar3 = fig.colorbar(im3, ax=axs[2], orientation='vertical', label='Temperature (K)')

    # 3x3滤波后的差异图像
    im4 = axs[3].imshow(filtered_diff_data, origin='lower', aspect='auto',
                        extent=[velocities[0], velocities[-1], dec_deg.min(), dec_deg.max()],
                        cmap='seismic', vmin=-6, vmax=6)
    axs[3].set_title(f"Filtered Difference Data at RA {ra_deg:.4f} degree")
    axs[3].set_xlabel("Velocity (km/s)")
    axs[3].set_ylabel("Dec (degrees)")
    cbar4 = fig.colorbar(im4, ax=axs[3], orientation='vertical', label='Temperature (K)')

    plt.savefig(f"{output_path}/slice_xch{x_index:03d}_ra{ra_deg:.4f}_filtered_diff.png")
    plt.close()
    print(f"Saved: slice_xch{x_index:03d}_ra{ra_deg:.4f}_filtered_diff.png")


fits_name = "/Users/naoj306/Desktop/DB/SFP/AquilaRift_12CO_21.7arcsec_vel0.1_sph_v1.0.fits"
output_path = "/Users/naoj306/Desktop/find_code/"
x_index = 360
generate_and_save_yv_slices_for_specific_x(fits_name, output_path, x_index)


Saved: slice_xch360_ra277.5127_filtered_diff.png
